In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.7.1'

# 3.2  MNIST数据集手写数字识别

## 3.2.1  数据集介绍
MNIST 包括6万张28x28的训练样本，1万张测试样本，很多教程都会对它”下手”几乎成为一个 “典范”，可以说它就是计算机视觉里面的Hello World。所以我们这里也会使用MNIST来进行实战。

前面在介绍卷积神经网络的时候说到过LeNet-5，LeNet-5之所以强大就是因为在当时的环境下将MNIST数据的识别率提高到了99%，这里我们也自己从头搭建一个卷积神经网络，也达到99%的准确率

## 3.2.2 手写数字识别
首先，我们定义一些超参数

In [4]:
BATCH_SIZE=512 #大概需要2G的显存
EPOCHS=20 # 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多

因为Pytorch里面包含了MNIST的数据集，所以我们这里直接使用即可。
如果第一次执行会生成data文件夹，并且需要一些时间下载，如果以前下载过就不会再次下载了

由于官方已经实现了dataset，所以这里可以直接使用DataLoader来对数据进行读取

In [5]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

测试集

In [6]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

8192it [00:17, 455.91it/s]              
1654784it [00:30, 86374.69it/s]                             

下面我们定义一个网络，网络包含两个卷积层，conv1和conv2，然后紧接着两个线性层作为输出，最后输出10个维度，这10个维度我们作为0-9的标识来确定识别出的是那个数字

在这里建议大家将每一层的输入和输出维度都作为注释标注出来，这样后面阅读代码的会方便很多

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
        self.conv1 = nn.Conv2d(1, 10, 5) # 输入通道数1，输出通道数10，核的大小5
        self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道数10，输出通道数20，核的大小3
        # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
        self.fc1 = nn.Linear(20*10*10, 500) # 输入通道数是2000，输出通道数是500
        self.fc2 = nn.Linear(500, 10) # 输入通道数是500，输出通道数是10，即10分类
    def forward(self,x):
        in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
        out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
        out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = F.relu(out) # batch*20*10*10
        out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10）
        out = self.fc1(out) # batch*2000 -> batch*500
        out = F.relu(out) # batch*500
        out = self.fc2(out) # batch*500 -> batch*10
        out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

我们实例化一个网络，实例化后使用.to方法将网络移动到GPU

优化器我们也直接选择简单暴力的Adam

In [8]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

下面定义一下训练的函数，我们将训练的所有操作都封装到这个函数中

In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

测试的操作也一样封装成一个函数

In [10]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

下面开始训练，这里就体现出封装起来的好处了，只要写两行就可以了

In [11]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.330203
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.206431
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.138403

Test set: Average loss: 0.1154, Accuracy: 9635/10000 (96%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.080030
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.086051
1654784it [00:57, 28844.02it/s]
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.066842

Test set: Average loss: 0.0594, Accuracy: 9805/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.040652
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.070239
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.052234

Test set: Average loss: 0.0413, Accuracy: 9864/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.032945
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.051212
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.067839

Test set: Average loss: 0.0500, Accuracy: 9839/10000 (98%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.029273
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.024207
Train Epoch: 5 [45568/60

我们看一下结果，准确率99%，没问题

如果你的模型连MNIST都搞不定，那么你的模型没有任何的价值

即使你的模型搞定了MNIST，你的模型也可能没有任何的价值

MNIST是一个很简单的数据集，由于它的局限性只能作为研究用途，对实际应用带来的价值非常有限。但是通过这个例子，我们可以完全了解一个实际项目的工作流程

我们找到数据集，对数据做预处理，定义我们的模型，调整超参数，测试训练，再通过训练结果对超参数进行调整或者对模型进行调整。

并且通过这个实战我们已经有了一个很好的模板，以后的项目都可以以这个模板为样例